In [1]:
wd <- dirname(getwd())
source(paste0(wd,"/mission_control/treasure_map.R"))

library(tidyr)
library(dplyr)

Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



#### 0 - Read and pre-process the data 

In [2]:
clinical <- (
    read.csv( paste0( TMP_DIR,"clinical_ready.csv"), stringsAsFactors = FALSE)
        %>% filter(clinical_cpi_mechanism2 %in% c("PD", "multiple"))    
)

In [28]:
drivers_purple <- read.csv( paste0( TMP_DIR,"drivers_DB_purple.csv"), stringsAsFactors = FALSE)
drivers <- (drivers_purple 
                %>% filter(driverLikelihood > .999) 
                %>% transmute( sampleId, driver = paste0(gene,"_",driver, "_", likelihoodMethod))
                %>% distinct())

In [11]:
#drivers_purple %>% head()

In [29]:
clinical_drivers <- clinical %>% left_join( drivers, by = "sampleId")

In [30]:
clinical_drivers %>% filter(grepl("B2M", driver)) %>% group_by(driver) %>% summarise(ct = n()) %>% arrange(desc(ct))

driver,ct
B2M_MUTATION_BIALLELIC,7
B2M_DEL_DEL,2
B2M_MUTATION_HOTSPOT,1


In [17]:
#common_drivers 

In [7]:
#drivers_purple %>% filter(gene == "B2M")

In [23]:
common_drivers <- (
    clinical_drivers
        %>% group_by(driver) 
        %>% summarise(ct = n()) 
        %>% arrange(desc(ct)) 
        %>% filter(ct > 5)
        %>% filter(grepl("BIALLELIC", driver))
        %>% select(driver, ct)
)
#sampleIds <- unique(drivers %>% pull(sampleId))

In [24]:
common_drivers

driver,ct
TP53_MUTATION_BIALLELIC,46
RB1_MUTATION_BIALLELIC,30
PBRM1_MUTATION_BIALLELIC,19
CDKN2A_MUTATION_BIALLELIC,17
KDM6A_MUTATION_BIALLELIC,15
STK11_MUTATION_BIALLELIC,15
PTEN_MUTATION_BIALLELIC,14
ARID1A_MUTATION_BIALLELIC,11
ARID2_MUTATION_BIALLELIC,10
BAP1_MUTATION_BIALLELIC,10


#### 1 - Run it

In [30]:
genes <- list()
genes[["sampleId"]] <- sampleIds
for (gene in common_drivers){
    cts <- c()
    for (sample in sampleIds){
        cts <- c(cts, nrow(drivers %>% filter(sampleId == sample, driver == gene)))
    }
    genes[[paste0("driver_",gene)]] <- cts
}

In [5]:
write.csv( data.frame(genes), file = paste0( TMP_DIR, "driver_ready.csv"), row.names=FALSE)